In [ ]:
%%time
!pip uninstall fastai en-core-web-sm en-core-web-lg spacy -y -q

In [ ]:
%%time
!pip install ../input/spacy3/catalogue-2.0.3-py3-none-any.whl ../input/spacy3/typer-0.3.2-py3-none-any.whl ../input/spacy3/srsly-2.4.1-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/pathy-0.5.2-py3-none-any.whl ../input/spacy3/smart_open-3.0.0-py3-none-any.whl ../input/spacy3/pydantic-1.7.3-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/thinc-8.0.3-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/spacy-3.0.6-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/spacy_legacy-3.0.5-py2.py3-none-any.whl -q

In [ ]:
%%time
!pip install ../input/spacy3/catalogue-2.0.3-py3-none-any.whl ../input/spacy3/typer-0.3.2-py3-none-any.whl ../input/spacy3/srsly-2.4.1-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/pathy-0.5.2-py3-none-any.whl ../input/spacy3/smart_open-3.0.0-py3-none-any.whl ../input/spacy3/pydantic-1.7.3-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/thinc-8.0.3-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/spacy-3.0.6-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/spacy_legacy-3.0.5-py2.py3-none-any.whl -q

In [ ]:
%%time
!pip install ../input/spacy3/en_core_web_lg-3.0.0-py3-none-any.whl ../input/spacy3/en_core_web_md-3.0.0-py3-none-any.whl ../input/spacy3/en_core_web_sm-3.0.0-py3-none-any.whl -q

In [ ]:
%%time
!pip install ../input/spacy3/spacy_alignments-0.8.3-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/spacy_transformers-1.0.2-py2.py3-none-any.whl ../input/spacy3/en_core_web_trf-3.0.0-py3-none-any.whl -q

In [ ]:
import spacy
assert spacy.__version__ == '3.0.6'

In [ ]:
import pandas as pd
import warnings
from itertools import chain
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import itertools
import os
import re
import json
import glob
from pathlib import Path
import numpy as np
import nltk
from tqdm.autonotebook import tqdm
nlp = spacy.load('en_core_web_lg')
nlp.max_length = 10000000
%matplotlib inline

In [ ]:
# test_files_path =  '../input/coleridgeinitiative-show-us-the-data/train'
# submission_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
# submission_df = submission_df.sample(n=10)


In [ ]:
test_files_path = '../input/coleridgeinitiative-show-us-the-data/test'
submission_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')


In [ ]:
def read_append_return(filename, files_path = test_files_path, output='text'):
    """
    Function to read json file and then return the text data from them and append to the dataframe
    """
    json_path = os.path.join(files_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [ ]:
tqdm.pandas()   #tqdm is used to show any code running with a progress bar. 
submission_df['text'] = submission_df['Id'].progress_apply(read_append_return)

In [ ]:
def text_cleaning(text):
    '''
    Converts all text to lower case, Removes special charecters, emojis and multiple spaces
    text - Sentence that needs to be cleaned
    '''
    text = ''.join([k if k not in '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~' else ' ' for k in text ])
    text = re.sub('[^A-Za-z0-9.]+', ' ', str(text).lower()).strip()
    
    return text

In [ ]:
%%time
tqdm.pandas()
submission_df['text1'] = submission_df['text'].progress_apply(text_cleaning)

In [ ]:
output_dir=Path('../input/coleridgemodel2/models3')
print("Loading from",output_dir)
nlp2 = spacy.load(output_dir)

In [ ]:
def get_string_max_len(temp_df, grouped_indexes):
    filtered_indexes = []
    for _ in grouped_indexes:
        temp = [len(temp_df.loc[i, 'PredictionString']) for i in _]
        temp_dict = dict(zip(_,temp))
        res = max(temp_dict.items(), key=operator.itemgetter(1))[0]
        filtered_indexes.append(res)
    return filtered_indexes

def reduce_similar_preds(pred_list):
    temp_df = pd.DataFrame({'PredictionString': pred_list})
    # create graph objects
    def compare(tweet1, threshold=0.70):
    # compare tweets using Levenshtein distance (or whatever string comparison metric) 
        matches = temp_df['PredictionString'].apply(lambda tweet2:(Levenshtein.ratio(tweet1, tweet2) >= threshold))

        # get positive matches
        matches = matches[matches].index.tolist()

        # convert to list of tuples
        return [*zip(iter(matches[:-1]), iter(matches[1:]))]

    # create graph objects
    nodes = temp_df.index.tolist()
    edges = [*itertools.chain(*temp_df["PredictionString"].apply(compare))]

    # create graphs
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)

    # get connected component indexes
    grouped_indexes = [*nx.connected_components(G)]

    # get a random choice index from each group
#     filtered_indexes = [random.choice([*_]) for _ in grouped_indexes]
    filtered_indexes = get_string_max_len(temp_df, grouped_indexes)

    return temp_df.loc[filtered_indexes, 'PredictionString'].to_list()

In [ ]:
%%time
import Levenshtein
import networkx as nx
import operator
all_pred_list = []
all_Id_list = []
n = 999999

for i in tqdm(range(submission_df.shape[0])): 
#     If length of the text is more than 999999 break it into multiple chunks of size 999999
    if len(submission_df.loc[i]['text1']) > 999999:
        chunks = [submission_df.loc[i]['text1'][j:j+n] for j in range(0, len(submission_df.loc[0]['text1']), n)]
        pred_list = []
        for chunk in chunks:
            doc = nlp2(chunk)
            pred_list.append(list(set([str(e) for e in list(doc.ents)])))
        pred_list = list(set(chain.from_iterable(pred_list)))
    else:
        doc = nlp2(submission_df.loc[i]['text1'])
        pred_list = list(set([str(e) for e in list(doc.ents)]))

    if len(pred_list)>1:
        pred_list = reduce_similar_preds(pred_list)
        
    pred = '|'.join(pred_list)
    if not pred:
        pred = ''
    all_pred_list.append(pred)
    all_Id_list.append(submission_df.loc[i]['Id'])

out_df = pd.DataFrame({'Id': all_Id_list, 'PredictionString': all_pred_list})

In [ ]:
out_df.to_csv('submission.csv', index=False)